In [49]:
import pandas as pd
import os
import folium
from folium.plugins import HeatMap
from folium.plugins import MarkerCluster
from folium.plugins import FastMarkerCluster

In [76]:
download_maps = True

In [51]:
os.listdir('../data/processed')

['.gitkeep',
 'bares.csv',
 'crimes.csv',
 'crimes_no_sp.csv',
 'crimes_old.csv',
 'crimes_with_arar_sp.csv',
 'escolas_estaduais.csv',
 'escolas_municipais.csv',
 'escolas_particulares.csv',
 'outras_escolas.csv',
 'pontos_cobertos.csv',
 'pontos_onibus.csv',
 'praças.csv',
 'praças.xlsx',
 'restaurantes.csv',
 '~$praças.xlsx']

In [66]:
crimes = pd.read_csv('../data/processed/crimes.csv', index_col=None)
escolas_particulares = pd.read_csv('../data/processed/escolas_particulares.csv', index_col=None)
escolas_estaduais = pd.read_csv('../data/processed/escolas_estaduais.csv', index_col=None)
pracas = pd.read_excel('../data/processed/praças.xlsx', index_col=None)
restaurantes = pd.read_csv('../data/processed/restaurantes.csv', index_col=None)
escolas_municipais = pd.read_csv('../data/processed/escolas_municipais.csv', index_col=None)
outras_escolas = pd.read_csv('../data/processed/outras_escolas.csv', index_col=None)
bares = pd.read_csv('../data/processed/bares.csv', index_col=None)
pontos_onibus = pd.read_csv('../data/processed/pontos_onibus.csv', index_col=None)

In [54]:
def generateBaseMap(default_location=[-22.010147, -47.890706], default_zoom_start=13):
    base_map = folium.Map(location=default_location, zoom_start=default_zoom_start)
    return base_map

## Plotting Praças

In [55]:
map_pracas = generateBaseMap()
latitudes = list(pracas['LATITUDE'])
longitudes = list(pracas['LONGITUDE'])

cnt = 0
for lat, lng in zip(latitudes, longitudes):
    cnt += 1
    try:
        folium.Marker(location=[lat, lng], tooltip=[lat, lng]).add_to(map_pracas)
    except:
        print(f'{cnt} has no coordinates')

1 has no coordinates
3 has no coordinates
20 has no coordinates
22 has no coordinates
28 has no coordinates
38 has no coordinates
40 has no coordinates
43 has no coordinates
52 has no coordinates
54 has no coordinates
63 has no coordinates
66 has no coordinates
68 has no coordinates
69 has no coordinates
92 has no coordinates
95 has no coordinates
96 has no coordinates
97 has no coordinates
99 has no coordinates
104 has no coordinates
105 has no coordinates
106 has no coordinates


In [56]:
map_pracas

In [57]:
pracas_coordinates = pd.DataFrame(data=pracas)
try:
    pracas_coordinates = pracas_coordinates.drop(columns=['Nome', 'Descrição'])
except:
    print('Dropped already!')
    
pracas_coordinates = pracas_coordinates[pd.notnull(pracas_coordinates['LATITUDE']) & pd.notnull(pracas_coordinates['LONGITUDE'])]
pracas_heat_map = generateBaseMap()
HeatMap(data=pracas_coordinates[['LATITUDE', 'LONGITUDE']], radius=10, max_zoom=13).add_to(pracas_heat_map)

Dropped already!


In [58]:
pracas_heat_map

## Plotting Pontos de onibus

In [10]:
map_pontos = generateBaseMap()
latitudes = list(pontos_onibus['Lat'])
longitudes = list(pontos_onibus['Long'])

cnt = 0
for lat, lng in zip(latitudes, longitudes):
    cnt += 1
    try:
        folium.Marker(location=[lat, lng], tooltip=[lat, lng]).add_to(map_pontos)
    except:
        print(f'{cnt} has no coordinates')

In [11]:
map_pontos

In [12]:
pontos_heat_map = generateBaseMap()
HeatMap(data=pontos_onibus[['Lat', 'Long']], radius=10, max_zoom=13).add_to(pontos_heat_map)

In [13]:
pontos_heat_map

## Plotting Bares

In [14]:
map_bares = generateBaseMap()
latitudes = list(bares['LATITUDE'])
longitudes = list(bares['LONGITUDE'])

cnt = 0
for lat, lng in zip(latitudes, longitudes):
    cnt += 1
    try:
        folium.Marker(location=[lat, lng], tooltip=[lat, lng]).add_to(map_bares)
    except:
        print(f'{cnt} has no coordinates')

In [15]:
map_bares

In [16]:
bares_heat_map = generateBaseMap()
HeatMap(data=bares[['LATITUDE', 'LONGITUDE']], radius=10, max_zoom=13).add_to(bares_heat_map)

In [17]:
bares_heat_map

## Plotting Resturantes

In [18]:
map_restaurantes = generateBaseMap()
latitudes = list(restaurantes['LATITUDE'])
longitudes = list(restaurantes['LONGITUDE'])

cnt = 0
for lat, lng in zip(latitudes, longitudes):
    cnt += 1
    try:
        folium.Marker(location=[lat, lng], tooltip=[lat, lng]).add_to(map_restaurantes)
    except:
        print(f'{cnt} has no coordinates')

In [19]:
map_restaurantes

In [20]:
restaurantes_heat_map = generateBaseMap()
HeatMap(data=restaurantes[['LATITUDE', 'LONGITUDE']], radius=10, max_zoom=13).add_to(restaurantes_heat_map)

In [21]:
restaurantes_heat_map

## Plotting everything

In [22]:
everything = pd.DataFrame(data=pontos_onibus)
everything = everything.rename(columns={'Lat' : 'LATITUDE', 'Long' : 'LONGITUDE'})
everything = pd.concat([everything, pracas_coordinates, bares, restaurantes], sort=False)
everything = everything[['LATITUDE', 'LONGITUDE']]
everything

,LATITUDE,LONGITUDE
0,-21.936774,-47.895920
1,-21.936067,-47.895156
2,-21.934742,-47.893912
3,-21.934391,-47.893440
4,-21.932259,-47.891519
...,...,...
259,-22.030438,-47.878881
260,-22.015727,-47.891877
261,-22.010371,-47.892051
262,-22.035546,-47.891158


In [23]:
all_heat_map = generateBaseMap()
HeatMap(data=everything,radius=10, max_zoom=13).add_to(all_heat_map)

In [24]:
all_heat_map

## Plotting Crimes

In [67]:
crimes_heat_map = generateBaseMap()
HeatMap(data=crimes[['LATITUDE', 'LONGITUDE']],radius=10, max_zoom=13).add_to(crimes_heat_map)

In [68]:
crimes_heat_map

In [75]:
crimes_clusters_map = generateBaseMap()
latitudes = list(crimes['LATITUDE'])
longitudes = list(crimes['LONGITUDE'])

callback = ('function (row) {'
                'var circle = L.circle(new L.LatLng(row[0], row[1]));'
                'return circle};')

crimes_clusters_map.add_child(FastMarkerCluster(crimes[['LATITUDE', 'LONGITUDE']].values.tolist(), callback=callback))
crimes_clusters_map

## Dowload html

In [77]:
if download_maps:
    # Praças
 #   map_pracas.save(outfile='maps/Pracas_points.html')
 #   pracas_heat_map.save(outfile='maps/Pracas_heatMap.html') 

    # Pontos de onibus
 #   map_pontos.save(outfile='maps/PontosOnibus_points.html')
 #   pontos_heat_map.save(outfile='maps/PontosOnibus_heatMap.html')
    
    # Bares
 #   map_bares.save(outfile='maps/Bares_points.html')
 #   bares_heat_map.save(outfile='maps/Bares_heatMap.html')
    
    # Restaurantes
 #   map_restaurantes.save(outfile='maps/Restaurantes_points.html')
 #   restaurantes_heat_map.save(outfile='maps/Restaurantes_heatMap.html')
    
    # Everything
 #   all_heat_map.save(outfile='maps/Everything.html')
    
    # Crimes clusters
    crimes_clusters_map.save(outfile='Crimes_clusters.html')